# Применение ResNet для распознавания людей в медицинской маске и без неее

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import files
# Скачайте датасет с Kaggle
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset
# Распакуйте скачанный архив
!unzip face-mask-12k-images-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/ashishjangra27/face-mask-12k-images-dataset
License(s): CC0-1.0
 98% 324M/330M [00:02<00:00, 98.7MB/s]
100% 330M/330M [00:02<00:00, 123MB/s] 
Archive:  face-mask-12k-images-dataset.zip
replace Face Mask Dataset/Test/WithMask/1163.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Путь к распакованному датасету
dataset_path = 'Face Mask Dataset/'

# для увеличения данных и разделения на обучающую и валидационную выборки.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

# train_generator validation_generator  для аугментации можно не использовать
train_generator = train_datagen.flow_from_directory(
    os.path.join(dataset_path, 'Train'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    os.path.join(dataset_path, 'Train'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation')


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
# Загрузка предобученной модели ResNet50 без верхних слоев
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Добавление нового выходного слоя
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Замораживание начальных слоев модели ResNet50
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Обучение модели
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs = 10)


Epoch 1/10
250/250 [==============================] - 150s 588ms/step - loss: 0.6653 - accuracy: 0.5890 - val_loss: 0.6418 - val_accuracy: 0.6522
Epoch 2/10
250/250 [==============================] - 145s 580ms/step - loss: 0.6312 - accuracy: 0.6526 - val_loss: 0.6173 - val_accuracy: 0.6885
Epoch 3/10
250/250 [==============================] - 145s 581ms/step - loss: 0.6088 - accuracy: 0.6848 - val_loss: 0.6005 - val_accuracy: 0.7117
Epoch 4/10
250/250 [==============================] - 130s 519ms/step - loss: 0.5945 - accuracy: 0.7004 - val_loss: 0.5898 - val_accuracy: 0.6966
Epoch 5/10
250/250 [==============================] - 145s 581ms/step - loss: 0.5825 - accuracy: 0.7055 - val_loss: 0.5767 - val_accuracy: 0.7132
Epoch 6/10
250/250 [==============================] - 129s 516ms/step - loss: 0.5702 - accuracy: 0.7203 - val_loss: 0.5718 - val_accuracy: 0.7550
Epoch 7/10
250/250 [==============================] - 147s 587ms/step - loss: 0.5583 - accuracy: 0.7376 - val_loss: 0.5645 -

In [ ]:
# Оценка модели
loss, accuracy = model.evaluate(validation_generator)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

63/63 [==============================] - 27s 420ms/step - loss: 0.5302 - accuracy: 0.7945
Loss: 0.5301723480224609
Accuracy: 0.7944999933242798
